In [1]:
#Assginment 1: Fast Trajectory Replanning
#CS 440: Intro to Artificial Intelligence
#Due date: 26th Feburary 2018
#authors: Ayush Joshi, Nikhil Mathur, and Dan Snyder
import heapq
import pygame
import random
import os
import math
import time
from random import *
from random import randint
from random import randrange
from ast import literal_eval as make_tuple
from IPython.core.debugger import set_trace

In [2]:
#Convention for the Environment
# Initialize Grid
# 0 is blocked and 1 is unblocked for track_maze
# 0 is unvisited and 1 is visited for track_maze
# gn = get neighbour
# nn = new neighbour
# p = parent

In [3]:
# Initialize Variables

blockwidth = 8  # Drawing dimensions of block
GridCols = 101   # No of columns
GridRows = 101   # No of rows

maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
track_maze = [[0 for y in range(GridRows)] for x in range(GridCols)]
first_parent_x = 0
first_parent_y = 0
last_parent_x = 0
last_parent_y = 0
cursor_x = 0
cursor_y = 0
i = 0

#main tree
tree = {}

In [4]:
#get the string for the cell location
def get_String(x,y):
    return str(x)+","+str(y)

#get the coordinates for the cell.
def get_coord(cell):
    li = cell.split(",")
    return int(li[0]),int(li[1])

#check if the cell is visited or not.
def is_visited(x,y):
    if x is None and y is None:
        return False
    x= int(x)
    y=int(y)
    if track_maze[x][y] == 1:
        return True
    return False

def is_first(x,y):
    if x == first_parent_x and y == first_parent_y:
        return True
    else:
        return False

# get the key for the value from the tree
def get_key(value):
    key_value_dict = {}
    for i in tree.items():
        a = i[0]
        b = i[1]
        if a and b:
            for item in b:
                key_value_dict[str(item)] = str(a)
    return key_value_dict[value]

In [5]:
def valid_coordinates(x,y):
    if x is None and y is None:
        return False
    if 0<= x <GridCols and 0<= y <GridRows:
        return True
    return False
    
#get the neighbours for a cell.
def get_neighbour(x,y,point):
    x= int(x)
    y= int(y)
    point = int(point)
    if point == 0 and valid_coordinates(x-1,y):
        return x-1,y
    if point == 1 and valid_coordinates(x+1,y):
        return x+1,y
    if point == 2 and valid_coordinates(x,y-1):
        return x,y-1
    if point == 3 and valid_coordinates(x,y+1):
        return x,y+1
    return None,None

#check if the cell has neighbours that are not visited.
def validate_neighbour(x,y):
    x=int(x)
    y=int(y)
    tf_value = False    
        
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            tf_value = True
    except:
        pass  
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            tf_value = True
    except:
        pass
        
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            tf_value = True
    except:
        pass
        
    return tf_value

#get the neighbour for a cell
def get_validate_neighbour(x,y):
    x = int(x)
    y = int(y)
    
    try:
        if valid_coordinates(x-1,y) and track_maze[x-1][y] == 0:
            return x-1, y
    except:
        pass
    
    try:
        if valid_coordinates(x+1,y) and track_maze[x+1][y] == 0:
            return x+1, y
    except:
        pass
    
    try:
        if valid_coordinates(x,y-1) and track_maze[x][y-1] == 0:
            return x, y-1
    except:
        pass
    
    try:
        if valid_coordinates(x,y+1) and track_maze[x][y+1] == 0:
            return x, y+1
    except:
        pass
    
    return None, None


In [6]:
#build_tree returns last child which was visited
def build_tree(p_x,p_y):
    nn_x = None
    nn_y = None
    last_p_x = p_x
    last_p_y = p_y

    while validate_neighbour(p_x,p_y) is True:
        ra = randint(0,3)
        nn_x,nn_y = get_neighbour(p_x,p_y,ra)
        if nn_x is None or nn_y is None or is_visited(nn_x,nn_y):
            while validate_neighbour(p_x,p_y):
                ran = randint(0,3)
                nn_x,nn_y = get_neighbour(p_x,p_y,ran)
                if valid_coordinates(nn_x,nn_y):
                    if is_visited(nn_x,nn_y) == False:
                        break

        if valid_coordinates(nn_x,nn_y) and is_visited(nn_x,nn_y) == False:
            tree[get_String(p_x,p_y)] = [get_String(nn_x,nn_y)] 
            track_maze[int(nn_x)][int(nn_y)] = 1
            last_p_x = nn_x
            last_p_y = nn_y
            rand_block_unblock = randint(0,100)
            #if rand_block_unblock < 30:
                #track_maze[nn_x][nn_y] = 1 # to be changed to 0, using to track
            #else:
                #track_maze[nn_y][nn_y] = 1
            p_x,p_y = nn_x,nn_y
    return last_p_x, last_p_y


In [7]:
#back_track returns new child for a parent with unvisited cell
def back_track(last_x,last_y):
    child_x = last_x
    child_y = last_y
    while is_first(child_x,child_y) == False:
        parent = get_key(get_String(child_x,child_y))
        parent_x,parent_y = get_coord(parent)
        if validate_neighbour(parent_x,parent_y):
            nnn_x,nnn_y = get_validate_neighbour(parent_x,parent_y)
            if nnn_x is not None and nnn_y is not None and is_visited(nnn_x,nnn_y) == False:
                try:
                    newlist = []
                    for item in tree[get_String(parent_x,parent_y)]:
                        newlist.append(item)    
                    newlist.append(get_String(int(nnn_x),int(nnn_y)))
                    #update the list
                    tree[get_String(parent_x,parent_y)] = newlist
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
                except:
                    tree[get_String(parent_x,parent_y)] = [get_String(int(nnn_x),int(nnn_y))]
                    track_maze[int(nnn_x)][int(nnn_y)] = 1
                return get_String(int(nnn_x),int(nnn_y))
        child_x,child_y = parent_x,parent_y
    return None

In [8]:
# Draw Screen
def drawScreen(GridSurface):
    GameScreen.blit(GridSurface,(0,0))
    pygame.display.update()

In [9]:
# Draw Grid
def drawGrid(myGridSurface):
    myGridSurface.fill((255,255,255))
    for x in range(len(track_maze)):
        for y in range(len(track_maze[x])):
            if track_maze[x][y] == 0: 
                pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
                pygame.draw.rect(myGridSurface, (40,40,40), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
            elif track_maze[x][y] == 1: 
                pygame.draw.rect(myGridSurface, (255,255,255), (x*blockwidth+10,y*blockwidth+10,blockwidth,blockwidth), 0)
                pygame.draw.rect(myGridSurface, (100,100,100), (x*blockwidth+10,y*blockwidth+10,blockwidth+1,blockwidth+1), 1)
    myGridSurface = myGridSurface.convert()
    return myGridSurface

In [10]:
# Make the grid
#areacoordinates = makeGrid()

In [11]:
## For visual
pygame.init()
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())
myfont = pygame.font.SysFont("monospace", 15)
start_x,start_y,goal_x,goal_y = 0,0,0,0
#generateStartFinish()

In [12]:
# Make Random Grid Visual
def makeGrid_visual():
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        global first_parent_x
        first_parent_x = p_x
        global first_parent_y
        first_parent_y = p_y
        ra = randint(0,3)
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    print("first parent",first_parent_x,first_parent_y)
    print("first child", gn_x,gn_y)
    track_maze[int(p_x)][int(p_y)] = 1
    track_maze[int(gn_x)][int(gn_y)] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    
    while True:
        #add space for text later
        GameScreen = pygame.display.set_mode((GridCols*blockwidth+20,GridRows*blockwidth+20))
        GridSurface = pygame.Surface(GameScreen.get_size())
        GridSurface = drawGrid(GridSurface)
        
        final_path = []
        closed_list = []
        cell_costs = []
        priority_list = []
        heuristic_list = []
        path_cost = 0
        elapsed_time = 0
        drawmode = 0
        nodes_expanded = 0
        #print(GridSurface)
        drawScreen(GridSurface)
        #time.sleep(2)
        #pygame.quit()
        #i = i +1
        #print("Tree",tree)
        #print("\n")
        new_x,new_y = build_tree(p_x,p_y)
        global last_parent_x
        global last_parent_y
        if last_parent_x == new_x and last_parent_y == new_y:
            break
        last_parent_x = new_x
        last_parent_y = new_y
        print("back to main",new_x,new_y)
        if new_x == first_parent_x and new_y == first_parent_y:
            return None
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == first_parent_x and p_y == first_parent_y:
                    break 
            if new_parent is None:
                return None
    return None


In [13]:

# Make Random Grid Visual
def makeGrid():
    i = 0
    gn_x = None
    gn_y = None
    p_x = None
    p_y = None
    while True:
        if gn_x != None and gn_y != None:
            print("first")
            break
        p_x = randint(0,GridCols-1)
        p_y = randint(0,GridRows-1)
        global first_parent_x
        first_parent_x = p_x
        global first_parent_y
        first_parent_y = p_y
        ra = randint(0,3)
        gn_x,gn_y = get_neighbour(p_x,p_y,ra) #get neighbor
   
    track_maze[p_x][p_y] = 1
    track_maze[p_x][p_y] = 1
    track_maze[gn_x][gn_y] = 1
    track_maze[gn_x][gn_y] = 1
    tree[str(p_x)+","+str(p_y)] = [get_String(gn_x,gn_y)]
    p_x,p_y = gn_x,gn_y
    
    
    while True:
        i = i +1
        #print("Tree",tree)
        print("\n")
        new_x,new_y = build_tree(p_x,p_y)
        global last_parent_x
        global last_parent_y
        if last_parent_x == new_x and last_parent_y == new_y:
            break
        
        last_parent_x = new_x
        last_parent_y = new_y
        
        print("back to main",new_x,new_y)
        if valid_coordinates(int(new_x),int(new_y)):
            new_parent = back_track(new_x,new_y)
            print("Back track found", new_parent)
            if new_parent:
                new_px,new_py = get_coord(new_parent)
                p_x,p_y = new_px,new_py
                if p_x == first_parent_x and p_y == first_parent_y:
                    break 
            if new_parent is None:
                return None
    return None


In [14]:
makeGrid_visual()

first
first parent 2 50
first child 2 51
back to main 9 78
Back track found 9,76
back to main 21 77
Back track found 20,76
back to main 20 76
Back track found 19,74
back to main 21 63
Back track found 21,65
back to main 23 71
Back track found 24,70
back to main 24 70
Back track found 25,72
back to main 23 76
Back track found 24,78
back to main 27 80
Back track found 22,79
back to main 16 83
Back track found 14,84
back to main 13 85
Back track found 11,84
back to main 10 85
Back track found 10,83
back to main 9 83
Back track found 7,83
back to main 1 92
Back track found 3,92
back to main 2 90
Back track found 1,89
back to main 0 89
Back track found 0,87
back to main 2 83
Back track found 4,83
back to main 5 80
Back track found 7,81
back to main 6 84
Back track found 8,81
back to main 8 81
Back track found 7,77
back to main 7 77
Back track found 3,78
back to main 0 86
Back track found 4,85
back to main 4 85
Back track found 3,87
back to main 3 87
Back track found 4,89
back to main 6 91
B

back to main 60 82
Back track found 60,80
back to main 62 84
Back track found 66,83
back to main 71 80
Back track found 70,79
back to main 74 74
Back track found 72,74
back to main 71 74
Back track found 70,72
back to main 62 77
Back track found 63,76
back to main 62 74
Back track found 62,70
back to main 66 71
Back track found 67,72
back to main 67 72
Back track found 67,75
back to main 68 80
Back track found 67,81
back to main 66 81
Back track found 67,77
back to main 67 77
Back track found 64,76
back to main 64 72
Back track found 63,68
back to main 63 65
Back track found 61,66
back to main 61 68
Back track found 59,67
back to main 55 67
Back track found 57,67
back to main 58 67
Back track found 57,66
back to main 57 66
Back track found 58,63
back to main 59 63
Back track found 58,64
back to main 58 64
Back track found 65,66
back to main 65 66
Back track found 59,72
back to main 56 69
Back track found 59,69
back to main 59 70
Back track found 58,72
back to main 58 72
Back track foun

back to main 60 28
Back track found 62,28
back to main 63 31
Back track found 65,29
back to main 80 29
Back track found 78,29
back to main 81 34
Back track found 82,33
back to main 82 32
Back track found 84,34
back to main 86 35
Back track found 87,36
back to main 86 43
Back track found 86,46
back to main 88 46
Back track found 84,47
back to main 85 48
Back track found 83,50
back to main 77 51
Back track found 80,50
back to main 80 50
Back track found 79,46
back to main 79 46
Back track found 78,45
back to main 77 45
Back track found 75,49
back to main 75 49
Back track found 73,46
back to main 71 45
Back track found 74,42
back to main 84 44
Back track found 81,48
back to main 81 45
Back track found 83,43
back to main 81 43
Back track found 79,43
back to main 78 43
Back track found 80,40
back to main 79 37
Back track found 77,36
back to main 76 31
Back track found 75,29
back to main 70 28
Back track found 79,33
back to main 79 33
Back track found 75,34
back to main 74 34
Back track foun

back to main 96 39
Back track found 95,38
back to main 94 38
Back track found 99,46
back to main 99 46
Back track found 100,54
back to main 98 53
Back track found 96,53
back to main 91 63
Back track found 93,60
back to main 88 62
Back track found 89,64
back to main 81 68
Back track found 81,66
back to main 80 82
Back track found 82,82
back to main 82 78
Back track found 84,78
back to main 84 77
Back track found 83,75
back to main 84 73
Back track found 83,71
back to main 96 76
Back track found 94,76
back to main 85 83
Back track found 86,82
back to main 87 81
Back track found 89,79
back to main 86 72
Back track found 88,73
back to main 88 74
Back track found 87,75
back to main 87 75
Back track found 90,75
back to main 90 74
Back track found 90,76
back to main 90 77
Back track found 89,72
back to main 89 72
Back track found 88,79
back to main 88 79
Back track found 82,83
back to main 83 84
Back track found 84,85
back to main 100 80
Back track found 100,86
back to main 100 87
Back track 

back to main 47 92
Back track found 49,91
back to main 49 94
Back track found 45,92
back to main 43 91
Back track found 40,91
back to main 40 91
Back track found 40,86
back to main 40 86
Back track found 39,81
back to main 39 81
Back track found 40,79
back to main 42 80
Back track found 44,81
back to main 47 80
Back track found 47,75
back to main 46 70
Back track found 46,72
back to main 45 75
Back track found 48,71
back to main 48 73
Back track found 43,76
back to main 43 76
Back track found 44,73
back to main 44 73
Back track found 43,70
back to main 43 71
Back track found 42,69
back to main 41 69
Back track found 40,68
back to main 34 67
Back track found 41,67
back to main 42 65
Back track found 41,73
back to main 40 71
Back track found 40,74
back to main 40 74
Back track found 38,72
back to main 37 72
Back track found 35,77
back to main 33 81
Back track found 34,80
back to main 34 80
Back track found 34,78
back to main 34 77
Back track found 36,79
back to main 36 78
Back track foun

back to main 25 84
Back track found 22,87
back to main 22 87
Back track found 19,88
back to main 19 86
Back track found 17,88
back to main 18 88
Back track found 17,90
back to main 16 90
Back track found 15,91
back to main 14 91
Back track found 18,96
back to main 19 96
Back track found 18,92
back to main 17 92
Back track found 20,100
back to main 16 97
Back track found 14,100
back to main 9 100
Back track found 20,99
back to main 20 99
Back track found 8,85
back to main 8 84
Back track found 11,80
back to main 11 83
Back track found 12,81
back to main 12 81
Back track found 19,79
back to main 21 79
Back track found 19,78
back to main 17 78
Back track found 24,73
back to main 24 73
Back track found 18,74
back to main 18 74
Back track found None


In [15]:
pygame.quit()

In [ ]:
#print(get_key("46,2"))
print(first_parent_x, first_parent_y)
#back_track(46,2)
print(back_track(2,2))


In [ ]:
track_maze

In [ ]:
# Main loop
running = True
pygame.init()
myfont = pygame.font.SysFont("monospace", 15)
GameScreen = pygame.display.set_mode((GridCols*blockwidth+200,GridRows*blockwidth+34))
GridSurface = pygame.Surface(GameScreen.get_size())

time.sleep(2)
GridSurface = drawGrid(GridSurface)
start_x,start_y,goal_x,goal_y = generateStartFinish()
final_path = []
closed_list = []
cell_costs = []
priority_list = []
heuristic_list = []
path_cost = 0
elapsed_time = 0
drawmode = 0
nodes_expanded = 0

MySearch = AStarSearch() # Initialize Object

while(running):
	# Get Input
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			running = False
		elif event.type == pygame.KEYDOWN:
			if event.key == pygame.K_ESCAPE:
				running = False
			elif event.key == pygame.K_g:
				track_maze = [['1' for y in range(GridRows)] for x in range(GridCols)]
				areacoordinates = makeGrid()
				GridSurface = drawGrid(GridSurface)
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new map")
			elif event.key == pygame.K_e:
				start_x,start_y,goal_x,goal_y = generateStartFinish()
				final_path = []
				closed_list = []
				priority_list = []
				heuristic_list = []
				cell_costs = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				print("Generated new start and finish points")
			elif event.key == pygame.K_s:
				# Save map: get filename
				filename = raw_input("Save map to: ")
				with open(os.path.join("./gen",filename),"w") as mapfile:
					mapfile.write(str((start_x,start_y)))		# Write start
					mapfile.write("\n")
					mapfile.write(str((goal_x,goal_y)))			# Write goal
					mapfile.write("\n")
					
					for area in areacoordinates:				# Write hard to traverse area centers
						mapfile.write(str((area[0],area[1])))		
						mapfile.write("\n")
					
					for y in range(len(track_maze[x])):				# Write each cell
						for x in range(len(track_maze)):					
							mapfile.write(str(track_maze[x][y]))		
						mapfile.write("\n")
					
					mapfile.close()
				print("Map saved!")
			elif event.key == pygame.K_l:
				# Load map: get filename
				filename = raw_input("Load map from: ")
				with open(os.path.join("./gen",filename),"r") as mapfile: #changed to allow using /maps folder
					new_start = make_tuple(mapfile.readline())
					start_x = new_start[0]
					start_y = new_start[1]
					new_goal = make_tuple(mapfile.readline())
					goal_x = new_goal[0]
					goal_y = new_goal[1]
					
					areacoordinates = []
					
					for i in range(8):
						new_area = make_tuple(mapfile.readline())
						areacoordinates.append((new_area[0],new_area[1]))
					
					for y in range(len(track_maze[x])):				# Read each cell
						for x in range(len(track_maze)):					
							track_maze[x][y] = mapfile.read(1)
						mapfile.read(1)
					
					mapfile.close()
				final_path = []
				closed_list = []
				cell_costs = []
				priority_list = []
				heuristic_list = []
				path_cost = 0
				elapsed_time = 0
				nodes_expanded = 0
				GridSurface = drawGrid(GridSurface)
				print("Map loaded!")
			elif event.key == pygame.K_UP:
				if cursor_y-1 >= 0:
					cursor_y -= 1
			elif event.key == pygame.K_LEFT:
				if cursor_x-1 >= 0:
					cursor_x -= 1
			elif event.key == pygame.K_RIGHT:
				if cursor_x+1 < GridCols:
					cursor_x += 1
			elif event.key == pygame.K_DOWN:
				if cursor_y+1 < GridRows:
					cursor_y += 1
			elif event.key == pygame.K_v:
				# draw closed list
				if drawmode == 0:
					drawmode = 1
				else:
					drawmode = 0
			elif event.key == pygame.K_a:		# -------- A* Search --------
				choice = -1
				while int(choice) < 1 or int(choice) > 6:
					choice = raw_input ("Enter (1) for Manhattan distance, (2) for Euclidean distance, (3) for Octile distance, (4) for Chebyshev distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				MySearch = AStarSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,choice)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_u:		# -------- Uniform Cost Search --------
				MySearch = UniformCostSearch()
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y,1)
				elapsed_time = time.time() - start_time
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_w:		# -------- Weighted A* Search --------
				choice = -1 #heuristic choice
				weight = 0 #weight of heuristic

				while (int(choice) < 1 or int(choice) > 6):
					choice = raw_input("Enter (1) for Manhattan distance, (2) for Euclidean Distance, (3) for Octile Distance, (4) for Chebyshev Distance, (5) for Straight-Diagonal Distance, or (6) Best/Minimum of all: ")
				while float(weight) < 1:
					weight = raw_input("Enter the selected weight for Weighted A* - must be >= 1: ")
				
				MySearch = WeightedAStarSearch(weight)
				start_time = time.time()
				closed_list, cell_costs, final_path, path_cost, priority_list, heuristic_list = MySearch.Search(start_x, start_y, goal_x, goal_y, choice)
				elapsed_time = time.time() - start_time
				
				nodes_expanded = len(closed_list)
			elif event.key == pygame.K_q:		# -------- Sequential A* Search --------
				MySearch = AStarSearch()

				
		#print(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)
		drawScreen(GridSurface,closed_list,final_path,path_cost,nodes_expanded,drawmode,elapsed_time,cell_costs, priority_list, heuristic_list)

pygame.quit()